In [4]:
from ultralytics import YOLO
import cv2
import os
from pathlib import Path


# 定义预测函数
def batch_predict_and_save(model, input_dir, output_label_dir):
    # 创建输出目录
    output_label_dir = Path(output_label_dir)
    output_label_dir.mkdir(parents=True, exist_ok=True)

    # 遍历输入目录中的所有 PNG 图像文件
    for img_path in Path(input_dir).glob("*.png"):
        # 使用模型进行预测
        results = model(img_path)

        # 提取图像名（不带扩展名）用于保存
        img_name = img_path.stem

        # 读取图像尺寸
        img = cv2.imread(str(img_path))
        h, w = img.shape[:2]

        # 保存预测框信息到文件
        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()  # 提取边界框 (xyxy 格式)
            for box in boxes:
                # 计算归一化中心点坐标和宽高
                x_center = (box[0] + box[2]) / 2 / w
                y_center = (box[1] + box[3]) / 2 / h
                width = (box[2] - box[0]) / w
                height = (box[3] - box[1]) / h

                # 将预测结果保存到 txt 文件
                txt_file_path = output_label_dir / f"{img_name}.txt"
                with open(txt_file_path, "a") as f:
                    f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")


# 加载模型
model = YOLO('/export/home/daifang/ncr/Yolo11/ultralytics-main/ultralytics/cfg/models/11/yolo11_PPA_P2P5.yaml').load(
    '/export/home/daifang/ncr/Yolo11/ultralytics-main/runs/train/BONE_BOX_250117BESTTT/weights/best.pt')

# # 定义输出文件路径
# output_file = "model.txt"

# # 打开文件进行写入
# with open(output_file, 'w') as f:
#     # 重定向输出到文件
#     print(model, file=f)


# 批量预测并
batch_predict_and_save(
    model=model,
    input_dir="/export/home/daifang/ncr/Yolo11/ultralytics-main/testCT",
    output_label_dir="/export/home/daifang/ncr/Yolo11/ultralytics-main/testCT_result")


WARNING ⚠️ no model scale passed. Assuming scale='n'.
Forwarding C3k2_PPA with input shape: torch.Size([1, 32, 64, 64])
Forwarding C3k2_PPA with input shape: torch.Size([1, 64, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 128, 16, 16])
Forwarding C3k2_PPA with input shape: torch.Size([1, 256, 8, 8])
Forwarding C3k2_PPA with input shape: torch.Size([1, 384, 16, 16])
Forwarding C3k2_PPA with input shape: torch.Size([1, 256, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 192, 16, 16])
Forwarding C3k2_PPA with input shape: torch.Size([1, 384, 8, 8])
Forwarding C3k2_PPA with input shape: torch.Size([1, 128, 64, 64])
Transferred 869/869 items from pretrained weights

Forwarding C3k2_PPA with input shape: torch.Size([1, 32, 160, 160])
Forwarding C3k2_PPA with input shape: torch.Size([1, 64, 80, 80])
Forwarding C3k2_PPA with input shape: torch.Size([1, 128, 40, 40])
Forwarding C3k2_PPA with input shape: torch.Size([1, 256, 20, 20])
Forwarding C3k2_PPA with input

In [11]:
import cv2
import os

# 路径设置
img_dir = "/export/home/daifang/ncr/Yolo11/ultralytics-main/CT/"         # 原图文件夹路径
txt_dir = "/export/home/daifang/ncr/Yolo11/ultralytics-main/result/"         # 检测结果txt路径
save_dir = "/export/home/daifang/ncr/Yolo11/ultralytics-main/output/"    # 可视化输出路径
os.makedirs(save_dir, exist_ok=True)

# 遍历所有txt
for txt_name in os.listdir(txt_dir):
    if not txt_name.endswith(".txt"):
        continue
    
    img_name = txt_name.replace(".txt", ".png")
    img_path = os.path.join(img_dir, img_name)
    txt_path = os.path.join(txt_dir, txt_name)
    
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ 无法找到 {img_path}")
        continue
    h, w, _ = img.shape

    # 读取检测框
    with open(txt_path, "r") as f:
        lines = f.readlines()
    
    for line in lines:
        cls, x_c, y_c, bw, bh = map(float, line.strip().split())
        x1 = int((x_c - bw/2) * w)
        y1 = int((y_c - bh/2) * h)
        x2 = int((x_c + bw/2) * w)
        y2 = int((y_c + bh/2) * h)

        box_w = x2 - x1
        box_h = y2 - y1

        # 绘制矩形与尺寸
        cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(img, f"{int(box_w)}x{int(box_h)}", (x1, y1-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1)
    
    save_path = os.path.join(save_dir, img_name)
    cv2.imwrite(save_path, img)
    print(f"✅ 保存可视化结果: {save_path}")


✅ 保存可视化结果: /export/home/daifang/ncr/Yolo11/ultralytics-main/output/1000001854_0026.png
✅ 保存可视化结果: /export/home/daifang/ncr/Yolo11/ultralytics-main/output/1000001854_0030.png
✅ 保存可视化结果: /export/home/daifang/ncr/Yolo11/ultralytics-main/output/1000001854_0031.png
✅ 保存可视化结果: /export/home/daifang/ncr/Yolo11/ultralytics-main/output/1000001854_0033.png
✅ 保存可视化结果: /export/home/daifang/ncr/Yolo11/ultralytics-main/output/1000001854_0039.png
✅ 保存可视化结果: /export/home/daifang/ncr/Yolo11/ultralytics-main/output/1000001854_0041.png
✅ 保存可视化结果: /export/home/daifang/ncr/Yolo11/ultralytics-main/output/1000001854_0042.png
✅ 保存可视化结果: /export/home/daifang/ncr/Yolo11/ultralytics-main/output/1000001854_0043.png
✅ 保存可视化结果: /export/home/daifang/ncr/Yolo11/ultralytics-main/output/1000001854_0045.png
✅ 保存可视化结果: /export/home/daifang/ncr/Yolo11/ultralytics-main/output/1000001854_0046.png
✅ 保存可视化结果: /export/home/daifang/ncr/Yolo11/ultralytics-main/output/1000001854_0047.png
✅ 保存可视化结果: /export/home/daifang/ncr/Yolo11/

In [5]:
import cv2
import os

# 输入文件夹
img_dir = "/export/home/daifang/ncr/Yolo11/ultralytics-main/CT"
txt_dir = "/export/home/daifang/ncr/Yolo11/ultralytics-main/result"
output_dir = "/export/home/daifang/ncr/Yolo11/ultralytics-main/output"

# 输出文件夹不存在则创建
os.makedirs(output_dir, exist_ok=True)

# 遍历所有图像
for img_file in os.listdir(img_dir):
    if not (img_file.endswith(".png") or img_file.endswith(".jpg")):
        continue

    # 获取文件名（不带后缀）
    base_name = os.path.splitext(img_file)[0]
    img_path = os.path.join(img_dir, img_file)
    txt_path = os.path.join(txt_dir, base_name + ".txt")

    # 如果没有对应的txt则跳过
    if not os.path.exists(txt_path):
        print(f"⚠️ No txt file for {img_file}, skipped.")
        continue

    # 读取图像
    img = cv2.imread(img_path)
    H, W = img.shape[:2]

    # 读取 YOLO txt 文件
    with open(txt_path, "r") as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue

        class_id, x_center, y_center, w, h = map(float, parts)

        # 转换为像素坐标
        x_center_pixel = int(x_center * W)
        y_center_pixel = int(y_center * H)
        w_pixel = int(w * W)
        h_pixel = int(h * H)

        x1 = int(x_center_pixel - w_pixel / 2)
        y1 = int(y_center_pixel - h_pixel / 2)
        x2 = int(x_center_pixel + w_pixel / 2)
        y2 = int(y_center_pixel + h_pixel / 2)

        # 绘制矩形框
        color = (0, 0, 255)  # 红色
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

        # # 绘制类别标签
        # cv2.putText(img, f"Class {int(class_id)}", (x1, max(y1 - 10, 0)),
        #             cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # 保存结果
    out_path = os.path.join(output_dir, img_file)
    cv2.imwrite(out_path, img)
    print(f"✅ Processed: {out_path}")

print("🎯 All images processed! Results saved in 'output/'")


⚠️ No txt file for 1000001854_0002.png, skipped.
⚠️ No txt file for 1000001854_0001.png, skipped.
⚠️ No txt file for 1000001854_0003.png, skipped.
⚠️ No txt file for 1000001854_0006.png, skipped.
⚠️ No txt file for 1000001854_0005.png, skipped.
⚠️ No txt file for 1000001854_0004.png, skipped.
⚠️ No txt file for 1000001854_0009.png, skipped.
⚠️ No txt file for 1000001854_0007.png, skipped.
⚠️ No txt file for 1000001854_0008.png, skipped.
⚠️ No txt file for 1000001854_0012.png, skipped.
⚠️ No txt file for 1000001854_0010.png, skipped.
⚠️ No txt file for 1000001854_0011.png, skipped.
⚠️ No txt file for 1000001854_0015.png, skipped.
⚠️ No txt file for 1000001854_0014.png, skipped.
⚠️ No txt file for 1000001854_0013.png, skipped.
⚠️ No txt file for 1000001854_0017.png, skipped.
⚠️ No txt file for 1000001854_0016.png, skipped.
⚠️ No txt file for 1000001854_0018.png, skipped.
⚠️ No txt file for 1000001854_0021.png, skipped.
⚠️ No txt file for 1000001854_0020.png, skipped.
⚠️ No txt file for 1

In [8]:
import cv2
import os

# 输入文件夹
img_dir = "/export/home/daifang/ncr/Yolo11/ultralytics-main/CT"
txt_dir = "/export/home/daifang/ncr/Yolo11/ultralytics-main/result"
output_dir = "/export/home/daifang/ncr/Yolo11/ultralytics-main/output"
crop_dir = os.path.join(img_dir, "crops")  # 在CT下新建crops文件夹

# 输出文件夹不存在则创建
os.makedirs(output_dir, exist_ok=True)
os.makedirs(crop_dir, exist_ok=True)

# 遍历所有图像
for img_file in os.listdir(img_dir):
    if not (img_file.endswith(".png") or img_file.endswith(".jpg")):
        continue

    # 获取文件名（不带后缀）
    base_name = os.path.splitext(img_file)[0]
    img_path = os.path.join(img_dir, img_file)
    txt_path = os.path.join(txt_dir, base_name + ".txt")

    # 如果没有对应的txt则跳过
    if not os.path.exists(txt_path):
        print(f"⚠️ No txt file for {img_file}, skipped.")
        continue

    # 读取图像
    img = cv2.imread(img_path)
    H, W = img.shape[:2]

    # 读取 YOLO txt 文件
    with open(txt_path, "r") as f:
        lines = f.readlines()

    crop_index = 1  # 每张图的裁剪编号

    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue

        class_id, x_center, y_center, w, h = map(float, parts)

        # 转换为像素坐标
        x_center_pixel = int(x_center * W)
        y_center_pixel = int(y_center * H)
        w_pixel = int(w * W)
        h_pixel = int(h * H)

        x1 = max(0, int(x_center_pixel - w_pixel / 2))
        y1 = max(0, int(y_center_pixel - h_pixel / 2))
        x2 = min(W, int(x_center_pixel + w_pixel / 2))
        y2 = min(H, int(y_center_pixel + h_pixel / 2))

        # 裁剪检测框区域（不带红框）
        crop_img = img[y1:y2, x1:x2]

        # 保存裁剪结果
        crop_path = os.path.join(crop_dir, f"{base_name}_crop{crop_index}.png")
        cv2.imwrite(crop_path, crop_img)
        print(f"✂️ Saved crop: {crop_path}")

        crop_index += 1

    # 保存一份原始图到 output 目录（不画框）
    out_path = os.path.join(output_dir, img_file)
    cv2.imwrite(out_path, img)
    print(f"✅ Saved original: {out_path}")

print("🎯 All images processed! Crops saved in 'CT/crops/', originals in 'output/'")


⚠️ No txt file for 1000001854_0002.png, skipped.
⚠️ No txt file for 1000001854_0001.png, skipped.
⚠️ No txt file for 1000001854_0003.png, skipped.
⚠️ No txt file for 1000001854_0006.png, skipped.
⚠️ No txt file for 1000001854_0005.png, skipped.
⚠️ No txt file for 1000001854_0004.png, skipped.
⚠️ No txt file for 1000001854_0009.png, skipped.
⚠️ No txt file for 1000001854_0007.png, skipped.
⚠️ No txt file for 1000001854_0008.png, skipped.
⚠️ No txt file for 1000001854_0012.png, skipped.
⚠️ No txt file for 1000001854_0010.png, skipped.
⚠️ No txt file for 1000001854_0011.png, skipped.
⚠️ No txt file for 1000001854_0015.png, skipped.
⚠️ No txt file for 1000001854_0014.png, skipped.
⚠️ No txt file for 1000001854_0013.png, skipped.
⚠️ No txt file for 1000001854_0017.png, skipped.
⚠️ No txt file for 1000001854_0016.png, skipped.
⚠️ No txt file for 1000001854_0018.png, skipped.
⚠️ No txt file for 1000001854_0021.png, skipped.
⚠️ No txt file for 1000001854_0020.png, skipped.
⚠️ No txt file for 1

In [6]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import torch, yaml, cv2, os, shutil, sys, copy
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
from tqdm import trange
from PIL import Image
from ultralytics import YOLO
from ultralytics.nn.tasks import attempt_load_weights
from ultralytics.utils.torch_utils import intersect_dicts
from ultralytics.utils.ops import xywh2xyxy, non_max_suppression
from pytorch_grad_cam import GradCAMPlusPlus, GradCAM, XGradCAM, EigenCAM, HiResCAM, LayerCAM, RandomCAM, EigenGradCAM, KPCA_CAM, AblationCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, scale_cam_image
from pytorch_grad_cam.activations_and_gradients import ActivationsAndGradients

def letterbox(im, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints
    shape = im.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better val mAP)
        r = min(r, 1.0)

    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding
    elif scaleFill:  # stretch
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  # width, height ratios

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        im = cv2.resize(im, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return im, ratio, (top, bottom, left, right)

class ActivationsAndGradients:
    """ Class for extracting activations and
    registering gradients from targetted intermediate layers """

    def __init__(self, model, target_layers, reshape_transform):
        self.model = model
        self.gradients = []
        self.activations = []
        self.reshape_transform = reshape_transform
        self.handles = []
        for target_layer in target_layers:
            self.handles.append(
                target_layer.register_forward_hook(self.save_activation))
            # Because of https://github.com/pytorch/pytorch/issues/61519,
            # we don't use backward hook to record gradients.
            self.handles.append(
                target_layer.register_forward_hook(self.save_gradient))

    def save_activation(self, module, input, output):
        activation = output

        if self.reshape_transform is not None:
            activation = self.reshape_transform(activation)
        self.activations.append(activation.cpu().detach())

    def save_gradient(self, module, input, output):
        if not hasattr(output, "requires_grad") or not output.requires_grad:
            # You can only register hooks on tensor requires grad.
            return

        # Gradients are computed in reverse order
        def _store_grad(grad):
            if self.reshape_transform is not None:
                grad = self.reshape_transform(grad)
            self.gradients = [grad.cpu().detach()] + self.gradients

        output.register_hook(_store_grad)

    def post_process(self, result):
        if self.model.end2end:
            logits_ = result[:, :, 4:]
            boxes_ = result[:, :, :4]
            sorted, indices = torch.sort(logits_[:, :, 0], descending=True)
            return logits_[0][indices[0]], boxes_[0][indices[0]]
        elif self.model.task == 'detect':
            logits_ = result[:, 4:]
            boxes_ = result[:, :4]
            sorted, indices = torch.sort(logits_.max(1)[0], descending=True)
            return torch.transpose(logits_[0], dim0=0, dim1=1)[indices[0]], torch.transpose(boxes_[0], dim0=0, dim1=1)[indices[0]]
        elif self.model.task == 'segment':
            logits_ = result[0][:, 4:4 + self.model.nc]
            boxes_ = result[0][:, :4]
            mask_p, mask_nm = result[1][2].squeeze(), result[1][1].squeeze().transpose(1, 0)
            c, h, w = mask_p.size()
            mask = (mask_nm @ mask_p.view(c, -1))
            sorted, indices = torch.sort(logits_.max(1)[0], descending=True)
            return torch.transpose(logits_[0], dim0=0, dim1=1)[indices[0]], torch.transpose(boxes_[0], dim0=0, dim1=1)[indices[0]], mask[indices[0]]
        elif self.model.task == 'pose':
            logits_ = result[:, 4:4 + self.model.nc]
            boxes_ = result[:, :4]
            poses_ = result[:, 4 + self.model.nc:]
            sorted, indices = torch.sort(logits_.max(1)[0], descending=True)
            return torch.transpose(logits_[0], dim0=0, dim1=1)[indices[0]], torch.transpose(boxes_[0], dim0=0, dim1=1)[indices[0]], torch.transpose(poses_[0], dim0=0, dim1=1)[indices[0]]
        elif self.model.task == 'obb':
            logits_ = result[:, 4:4 + self.model.nc]
            boxes_ = result[:, :4]
            angles_ = result[:, 4 + self.model.nc:]
            sorted, indices = torch.sort(logits_.max(1)[0], descending=True)
            return torch.transpose(logits_[0], dim0=0, dim1=1)[indices[0]], torch.transpose(boxes_[0], dim0=0, dim1=1)[indices[0]], torch.transpose(angles_[0], dim0=0, dim1=1)[indices[0]]
        elif self.model.task == 'classify':
            return result[0]
  
    def __call__(self, x):
        self.gradients = []
        self.activations = []
        model_output = self.model(x)
        if self.model.task == 'detect':
            post_result, pre_post_boxes = self.post_process(model_output[0])
            return [[post_result, pre_post_boxes]]
        elif self.model.task == 'segment':
            post_result, pre_post_boxes, pre_post_mask = self.post_process(model_output)
            return [[post_result, pre_post_boxes, pre_post_mask]]
        elif self.model.task == 'pose':
            post_result, pre_post_boxes, pre_post_pose = self.post_process(model_output[0])
            return [[post_result, pre_post_boxes, pre_post_pose]]
        elif self.model.task == 'obb':
            post_result, pre_post_boxes, pre_post_angle = self.post_process(model_output[0])
            return [[post_result, pre_post_boxes, pre_post_angle]]
        elif self.model.task == 'classify':
            data = self.post_process(model_output)
            return [data]

    def release(self):
        for handle in self.handles:
            handle.remove()

class yolo_detect_target(torch.nn.Module):
    def __init__(self, ouput_type, conf, ratio, end2end) -> None:
        super().__init__()
        self.ouput_type = ouput_type
        self.conf = conf
        self.ratio = ratio
        self.end2end = end2end
    
    def forward(self, data):
        post_result, pre_post_boxes = data
        result = []
        for i in trange(int(post_result.size(0) * self.ratio)):
            if (self.end2end and float(post_result[i, 0]) < self.conf) or (not self.end2end and float(post_result[i].max()) < self.conf):
                break
            if self.ouput_type == 'class' or self.ouput_type == 'all':
                if self.end2end:
                    result.append(post_result[i, 0])
                else:
                    result.append(post_result[i].max())
            elif self.ouput_type == 'box' or self.ouput_type == 'all':
                for j in range(4):
                    result.append(pre_post_boxes[i, j])
        return sum(result)

class yolo_segment_target(yolo_detect_target):
    def __init__(self, ouput_type, conf, ratio, end2end):
        super().__init__(ouput_type, conf, ratio, end2end)
    
    def forward(self, data):
        post_result, pre_post_boxes, pre_post_mask = data
        result = []
        for i in trange(int(post_result.size(0) * self.ratio)):
            if float(post_result[i].max()) < self.conf:
                break
            if self.ouput_type == 'class' or self.ouput_type == 'all':
                result.append(post_result[i].max())
            elif self.ouput_type == 'box' or self.ouput_type == 'all':
                for j in range(4):
                    result.append(pre_post_boxes[i, j])
            elif self.ouput_type == 'segment' or self.ouput_type == 'all':
                result.append(pre_post_mask[i].mean())
        return sum(result)


class yolo_pose_target(yolo_detect_target):
    def __init__(self, ouput_type, conf, ratio, end2end):
        super().__init__(ouput_type, conf, ratio, end2end)
    
    def forward(self, data):
        post_result, pre_post_boxes, pre_post_pose = data
        result = []
        for i in trange(int(post_result.size(0) * self.ratio)):
            if float(post_result[i].max()) < self.conf:
                break
            if self.ouput_type == 'class' or self.ouput_type == 'all':
                result.append(post_result[i].max())
            elif self.ouput_type == 'box' or self.ouput_type == 'all':
                for j in range(4):
                    result.append(pre_post_boxes[i, j])
            elif self.ouput_type == 'pose' or self.ouput_type == 'all':
                result.append(pre_post_pose[i].mean())
        return sum(result)

class yolo_obb_target(yolo_detect_target):
    def __init__(self, ouput_type, conf, ratio, end2end):
        super().__init__(ouput_type, conf, ratio, end2end)
    
    def forward(self, data):
        post_result, pre_post_boxes, pre_post_angle = data
        result = []
        for i in trange(int(post_result.size(0) * self.ratio)):
            if float(post_result[i].max()) < self.conf:
                break
            if self.ouput_type == 'class' or self.ouput_type == 'all':
                result.append(post_result[i].max())
            elif self.ouput_type == 'box' or self.ouput_type == 'all':
                for j in range(4):
                    result.append(pre_post_boxes[i, j])
            elif self.ouput_type == 'obb' or self.ouput_type == 'all':
                result.append(pre_post_angle[i])
        return sum(result)

class yolo_classify_target(yolo_detect_target):
    def __init__(self, ouput_type, conf, ratio, end2end):
        super().__init__(ouput_type, conf, ratio, end2end)
    
    def forward(self, data):
        return data.max()

class yolo_heatmap:
    def __init__(self, cfg, weight, device, method, layer, backward_type, conf_threshold, ratio, show_result, renormalize, task, img_size):
        device = torch.device(device)
        model_yolo = YOLO(cfg)  
        model_yolo.load(weight)  
        model_names = model_yolo.names
        print(f'model class info:{model_names}')
        model = copy.deepcopy(model_yolo.model)
        model.to(device)
        model.info()
        for p in model.parameters():
            p.requires_grad_(True)
        model.eval()
        
        model.task = task
        if not hasattr(model, 'end2end'):
            model.end2end = False
        
        if task == 'detect':
            target = yolo_detect_target(backward_type, conf_threshold, ratio, model.end2end)
        elif task == 'segment':
            target = yolo_segment_target(backward_type, conf_threshold, ratio, model.end2end)
        elif task == 'pose':
            target = yolo_pose_target(backward_type, conf_threshold, ratio, model.end2end)
        elif task == 'obb':
            target = yolo_obb_target(backward_type, conf_threshold, ratio, model.end2end)
        elif task == 'classify':
            target = yolo_classify_target(backward_type, conf_threshold, ratio, model.end2end)
        else:
            raise Exception(f"not support task({task}).")
        
        target_layers = [model.model[l] for l in layer]
        method = eval(method)(model, target_layers)
        method.activations_and_grads = ActivationsAndGradients(model, target_layers, None)
        
        colors = np.random.uniform(0, 255, size=(len(model_names), 3)).astype(np.int32)
        self.__dict__.update(locals())
    
    def post_process(self, result):
        result = non_max_suppression(result, conf_thres=self.conf_threshold, iou_thres=0.65)[0]
        return result

    def draw_detections(self, box, color, name, img):
        xmin, ymin, xmax, ymax = list(map(int, list(box)))
        cv2.rectangle(img, (xmin, ymin), (xmax, ymax), tuple(int(x) for x in color), 2) # 绘制检测框
        cv2.putText(img, str(name), (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, tuple(int(x) for x in color), 2, lineType=cv2.LINE_AA)  # 绘制类别、置信度
        return img

    def renormalize_cam_in_bounding_boxes(self, boxes, image_float_np, grayscale_cam):
        """Normalize the CAM to be in the range [0, 1] 
        inside every bounding boxes, and zero outside of the bounding boxes. """
        renormalized_cam = np.zeros(grayscale_cam.shape, dtype=np.float32)
        for x1, y1, x2, y2 in boxes:
            x1, y1 = max(x1, 0), max(y1, 0)
            x2, y2 = min(grayscale_cam.shape[1] - 1, x2), min(grayscale_cam.shape[0] - 1, y2)
            renormalized_cam[y1:y2, x1:x2] = scale_cam_image(grayscale_cam[y1:y2, x1:x2].copy())    
        renormalized_cam = scale_cam_image(renormalized_cam)
        eigencam_image_renormalized = show_cam_on_image(image_float_np, renormalized_cam, use_rgb=True)
        return eigencam_image_renormalized
    
    def process(self, img_path, save_path):
        # img process
        try:
            img = cv2.imdecode(np.fromfile(img_path, np.uint8), cv2.IMREAD_COLOR)
        except:
            print(f"Warning... {img_path} read failure.")
            return
        img, _, (top, bottom, left, right) = letterbox(img, new_shape=(self.img_size, self.img_size), auto=False) # 如果需要完全固定成宽高一样就把auto设置为False
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = np.float32(img) / 255.0
        tensor = torch.from_numpy(np.transpose(img, axes=[2, 0, 1])).unsqueeze(0).to(self.device)
        print(f'tensor size:{tensor.size()}')
        
        try:
            # 计算CAM
            print(f'self.method:{self.method}')
            print(f'self.target:{self.target}')
            grayscale_cam = self.method(tensor, [self.target])
        except AttributeError as e:
            print(f"Warning... self.method(tensor, [self.target]) failure.")
            return
        
        grayscale_cam = grayscale_cam[0, :]
        # 在热力图上应用双边滤波
        grayscale_cam = cv2.bilateralFilter(grayscale_cam, d=9, sigmaColor=100, sigmaSpace=100)

        
        cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
        
        pred = self.model_yolo.predict(tensor, conf=self.conf_threshold, iou=0.7)[0]
        if self.renormalize and self.task in ['detect', 'segment', 'pose']:
            cam_image = self.renormalize_cam_in_bounding_boxes(pred.boxes.xyxy.cpu().detach().numpy().astype(np.int32), img, grayscale_cam)
        if self.show_result:
            cam_image = pred.plot(img=cam_image,
                                  conf=False, # 显示置信度
                                  font_size=None, # 字体大小，None为根据当前image尺寸计算
                                  line_width=None, # 线条宽度，None为根据当前image尺寸计算
                                  labels=False, # 显示标签
                                  )
        
        # 去掉padding边界
        cam_image = cam_image[top:cam_image.shape[0] - bottom, left:cam_image.shape[1] - right]
        cam_image = Image.fromarray(cam_image)
        cam_image.save(save_path)
    
    def __call__(self, img_path, save_path):
        # remove dir if exist
        if os.path.exists(save_path):
            shutil.rmtree(save_path)
        # make dir if not exist
        os.makedirs(save_path, exist_ok=True)

        if os.path.isdir(img_path):
            for img_path_ in os.listdir(img_path):
                self.process(f'{img_path}/{img_path_}', f'{save_path}/{img_path_}')
        else:
            self.process(img_path, f'{save_path}/result.png')
        
def get_params():
    params = {
        'cfg': '/export/home/daifang/ncr/Yolo11/ultralytics-main/ultralytics/cfg/models/11/yolo11_PPA_P2P5.yaml',
        # 'weight': '/export/home/daifang/ncr/Yolo11/ultralytics-main/runs/train/BONE_BOX_250307/weights/last.pt', 
        'weight': '/export/home/daifang/ncr/Yolo11/ultralytics-main/runs/train/BONE_BOX_250117BESTTT/weights/best.pt',
        'device': 'cuda:0',
        'method': 'GradCAMPlusPlus', # GradCAMPlusPlus, GradCAM, XGradCAM, EigenCAM, HiResCAM, LayerCAM, RandomCAM, EigenGradCAM, KPCA_CAM
        'layer': [13,15,17,18, 20, 21, 22, 23, 24,26],
        'backward_type': 'all', # detect:<class, box, all> segment:<class, box, segment, all> pose:<box, keypoint, all> obb:<box, angle, all> classify:<all>
        'conf_threshold': 0.01, # 0.2
        'ratio': 0.02, # 0.02-0.1
        'show_result': False, # 不需要绘制结果请设置为False
        'renormalize': False, # 需要把热力图限制在框内请设置为True(仅对detect,segment,pose有效)
        'task':'detect', # 任务(detect,segment,pose,obb,classify)
        'img_size':512, # 图像尺寸
    }
    return params

# pip install grad-cam==1.5.4 --no-deps
if __name__ == '__main__':
    model = yolo_heatmap(**get_params())
    # model(r'/home/hjj/Desktop/dataset/dataset_coco/coco/images/val2017/000000361238.jpg', 'result')
    # model(r'H:\bone_metastasis\total_data\4class\4class\Osteolytic_deal\CT_Bone', 'Osteolytic_deal_result')
    # model('/export/home/daifang/ncr/data/normal_chest', '/export/home/daifang/ncr/data/normal_grad_result1')
    # model('/export/home/daifang/ncr/Yolo11/datasets_0422/images/train', '/export/home/daifang/ncr/CAM/result')
    model('/export/home/daifang/ncr/Yolo11/ultralytics-main/CT', '/export/home/daifang/ncr/CAM/result')


WARNING ⚠️ no model scale passed. Assuming scale='n'.
Forwarding C3k2_PPA with input shape: torch.Size([1, 32, 64, 64])
Forwarding C3k2_PPA with input shape: torch.Size([1, 64, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 128, 16, 16])
Forwarding C3k2_PPA with input shape: torch.Size([1, 256, 8, 8])
Forwarding C3k2_PPA with input shape: torch.Size([1, 384, 16, 16])
Forwarding C3k2_PPA with input shape: torch.Size([1, 256, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 192, 16, 16])
Forwarding C3k2_PPA with input shape: torch.Size([1, 384, 8, 8])
Forwarding C3k2_PPA with input shape: torch.Size([1, 128, 64, 64])
Transferred 869/869 items from pretrained weights
model class info:{0: '0'}
Forwarding C3k2_PPA with input shape: torch.Size([1, 32, 8, 8])
Forwarding C3k2_PPA with input shape: torch.Size([1, 64, 4, 4])
Forwarding C3k2_PPA with input shape: torch.Size([1, 128, 2, 2])
Forwarding C3k2_PPA with input shape: torch.Size([1, 256, 1, 1])
Forwarding C3k2

  0%|          | 0/435 [00:00<?, ?it/s]

Warning... self.method(tensor, [self.target]) failure.
tensor size:torch.Size([1, 3, 512, 512])
self.method:<pytorch_grad_cam.grad_cam_plusplus.GradCAMPlusPlus object at 0x7f3becc17d60>
self.target:yolo_detect_target()
Forwarding C3k2_PPA with input shape: torch.Size([1, 32, 128, 128])
Forwarding C3k2_PPA with input shape: torch.Size([1, 64, 64, 64])
Forwarding C3k2_PPA with input shape: torch.Size([1, 128, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 256, 16, 16])
Forwarding C3k2_PPA with input shape: torch.Size([1, 384, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 256, 64, 64])
Forwarding C3k2_PPA with input shape: torch.Size([1, 192, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 384, 16, 16])
Forwarding C3k2_PPA with input shape: torch.Size([1, 128, 128, 128])



  0%|          | 0/435 [00:00<?, ?it/s]


Warning... self.method(tensor, [self.target]) failure.
tensor size:torch.Size([1, 3, 512, 512])
self.method:<pytorch_grad_cam.grad_cam_plusplus.GradCAMPlusPlus object at 0x7f3becc17d60>
self.target:yolo_detect_target()
Forwarding C3k2_PPA with input shape: torch.Size([1, 32, 128, 128])
Forwarding C3k2_PPA with input shape: torch.Size([1, 64, 64, 64])
Forwarding C3k2_PPA with input shape: torch.Size([1, 128, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 256, 16, 16])
Forwarding C3k2_PPA with input shape: torch.Size([1, 384, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 256, 64, 64])
Forwarding C3k2_PPA with input shape: torch.Size([1, 192, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 384, 16, 16])
Forwarding C3k2_PPA with input shape: torch.Size([1, 128, 128, 128])


...

  5%|▍         | 20/435 [00:00<00:00, 14191.52it/s]


Forwarding C3k2_PPA with input shape: torch.Size([1, 32, 128, 128])
Forwarding C3k2_PPA with input shape: torch.Size([1, 64, 64, 64])
Forwarding C3k2_PPA with input shape: torch.Size([1, 128, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 256, 16, 16])
Forwarding C3k2_PPA with input shape: torch.Size([1, 384, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 256, 64, 64])
Forwarding C3k2_PPA with input shape: torch.Size([1, 192, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 384, 16, 16])
Forwarding C3k2_PPA with input shape: torch.Size([1, 128, 128, 128])
0: 512x512 4 0s, 30.7ms
Speed: 0.0ms preprocess, 30.7ms inference, 1.4ms postprocess per image at shape (1, 3, 512, 512)


tensor size:torch.Size([1, 3, 512, 512])
self.method:<pytorch_grad_cam.grad_cam_plusplus.GradCAMPlusPlus object at 0x7f3becc17d60>
self.target:yolo_detect_target()
Forwarding C3k2_PPA with input shape: torch.Size([1, 32, 128, 128])
Forwarding C3k2_PPA with input shape: torch.Size([1, 64, 64, 64])
Forwarding C3k2_PPA with input shape: torch.Size([1, 128, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 256, 16, 16])
Forwarding C3k2_PPA with input shape: torch.Size([1, 384, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 256, 64, 64])
Forwarding C3k2_PPA with input shape: torch.Size([1, 192, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 384, 16, 16])
Forwarding C3k2_PPA with input shape: torch.Size([1, 128, 128, 128])


  3%|▎         | 13/435 [00:00<00:00, 12114.19it/s]


Forwarding C3k2_PPA with input shape: torch.Size([1, 32, 128, 128])
Forwarding C3k2_PPA with input shape: torch.Size([1, 64, 64, 64])
Forwarding C3k2_PPA with input shape: torch.Size([1, 128, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 256, 16, 16])
Forwarding C3k2_PPA with input shape: torch.Size([1, 384, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 256, 64, 64])
Forwarding C3k2_PPA with input shape: torch.Size([1, 192, 32, 32])
Forwarding C3k2_PPA with input shape: torch.Size([1, 384, 16, 16])
Forwarding C3k2_PPA with input shape: torch.Size([1, 128, 128, 128])
0: 512x512 2 0s, 30.7ms
Speed: 0.0ms preprocess, 30.7ms inference, 1.4ms postprocess per image at shape (1, 3, 512, 512)


In [ ]:
from ultralytics import YOLO
import cv2
import os
from pathlib import Path


# 定义预测函数
def batch_predict_and_save(model, input_dir, output_base_dir, max_folders=None):
    # 创建输出目录的根路径
    output_base_dir = Path(output_base_dir)
    output_base_dir.mkdir(parents=True, exist_ok=True)

    # 获取输入目录中的所有病人ID文件夹
    patient_folders = [f for f in Path(input_dir).iterdir() if f.is_dir()]

    # 如果指定了max_folders，限制处理的文件夹数量
    if max_folders:
        patient_folders = patient_folders[:max_folders]

    # 遍历病人文件夹
    for patient_folder in patient_folders:
        # 创建该病人ID文件夹的输出目录
        patient_output_dir = output_base_dir / patient_folder.name
        patient_output_dir.mkdir(parents=True, exist_ok=True)

        # 创建保存预测框信息的 box_txt 文件夹
        box_txt_dir = patient_output_dir / "box_txt"
        box_txt_dir.mkdir(parents=True, exist_ok=True)

        # 遍历病人文件夹中的所有 PNG 图像文件
        for img_path in patient_folder.glob("*.png"):
            # 使用模型进行预测
            results = model(img_path)

            # 提取图像名（不带扩展名）用于保存
            img_name = img_path.stem

            # 读取图像尺寸
            img = cv2.imread(str(img_path))
            h, w = img.shape[:2]

            # 保存预测框信息到文件
            for result in results:
                boxes = result.boxes.xyxy.cpu().numpy()  # 提取边界框 (xyxy 格式)
                for box in boxes:
                    # 计算归一化中心点坐标和宽高
                    x_center = (box[0] + box[2]) / 2 / w
                    y_center = (box[1] + box[3]) / 2 / h
                    width = (box[2] - box[0]) / w
                    height = (box[3] - box[1]) / h

                    # 将预测结果保存到 txt 文件
                    txt_file_path = box_txt_dir / f"{img_name}.txt"
                    with open(txt_file_path, "a") as f:
                        f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")


# 加载模型
model = YOLO(r'H:\bone_metastasis\Yolov11\ultralytics-main\ultralytics-main\runs\train\0511_PPA_CAFM2\yolo11_CAFM_PPA_P2.yaml').load(
    r"H:\bone_metastasis\Yolov11\ultralytics-main\ultralytics-main\runs\train\0511_PPA_CAFM2\weights\best.pt")

# 定义输入目录和输出文件夹路径
input_dir = r"H:\bone_metastasis\total_data\train_data\CT_ORIGINAL_wuhead512"
output_base_dir = r"H:\bone_metastasis\Yolov11\ultralytics-main\ultralytics-main\result_predict"

# 设置要处理的病人文件夹的最大数量
max_folders = 100  # 设置为处理5个文件夹，您可以根据需要修改这个数字

# 批量预测并保存结果
batch_predict_and_save(
    model=model,
    input_dir=input_dir,
    output_base_dir=output_base_dir,
    max_folders=max_folders
)


WARNING ⚠️ no model scale passed. Assuming scale='n'.


KeyError: 'C3k2_PPA'

In [19]:
import shutil
from pathlib import Path
import os

def copy_files_based_on_box_img(result_predict_dir, red_pet_dir, ct_original_dir):
    # 获取 result_predict 中的所有病人ID文件夹
    patient_folders = [f for f in Path(result_predict_dir).iterdir() if f.is_dir()]

    # 遍历每个病人文件夹
    for patient_folder in patient_folders:
        # 获取病人ID，文件夹名就是病人ID
        patient_id = patient_folder.name
        
        # 创建 CT 和 RedPET 文件夹
        ct_folder = patient_folder / "CT"
        redpet_folder = patient_folder / "RedPET"
        
        # 如果没有这些文件夹则创建
        ct_folder.mkdir(parents=True, exist_ok=True)
        redpet_folder.mkdir(parents=True, exist_ok=True)

        # 遍历病人文件夹中的 box_img 文件夹中的所有 png 文件
        box_img_dir = patient_folder / "box_img"
        for img_path in box_img_dir.glob("*.png"):
            # 提取图像文件名（不带扩展名）
            img_name = img_path.stem
            
            # 生成目标路径：从 red_pet_dir 复制同名文件到 RedPET 文件夹
            corresponding_redpet_img_path = Path(red_pet_dir) / f"{img_name}.png"
            print(redpet_folder)
            if corresponding_redpet_img_path.exists():
                # 复制文件到 RedPET 文件夹
                shutil.copy(corresponding_redpet_img_path, redpet_folder / f"{img_name}.png")
                print(f"复制 {img_name}.png 到 RedPET 文件夹.")
            else:
                print(f"在 Red_PET_512all 中找不到图像: {img_name}.png, 跳过.")

        # 从 CT_ORIGINAL 中复制病人ID文件夹内容到 CT 文件夹
        corresponding_ct_folder = Path(ct_original_dir) / patient_id
        if corresponding_ct_folder.exists():
            # 遍历病人ID文件夹中的所有文件和子文件夹
            for item in corresponding_ct_folder.iterdir():
                # 复制文件或子文件夹
                if item.is_dir():
                    shutil.copytree(item, ct_folder / item.name)
                else:
                    shutil.copy(item, ct_folder / item.name)
            print(f"复制 {patient_id} 的 CT 文件夹内容到 CT 文件夹.")
        else:
            print(f"在 CT_ORIGINAL 中找不到病人ID文件夹: {patient_id}, 跳过.")

# 定义路径
result_predict_dir = "/export/home/daifang/ncr/data/pred_yolo_100/result_predict"
red_pet_dir = "/export/home/daifang/ncr/data/Red_PET_512all"
ct_original_dir = "/export/home/daifang/ncr/data/CT_ORIGINAL"

# 执行文件复制操作
copy_files_based_on_box_img(result_predict_dir, red_pet_dir, ct_original_dir)


/export/home/daifang/ncr/data/pred_yolo_100/result_predict/1000354500/RedPET
复制 1000354500_0077.png 到 RedPET 文件夹.
/export/home/daifang/ncr/data/pred_yolo_100/result_predict/1000354500/RedPET
复制 1000354500_0079.png 到 RedPET 文件夹.
/export/home/daifang/ncr/data/pred_yolo_100/result_predict/1000354500/RedPET
复制 1000354500_0083.png 到 RedPET 文件夹.
/export/home/daifang/ncr/data/pred_yolo_100/result_predict/1000354500/RedPET
复制 1000354500_0084.png 到 RedPET 文件夹.
/export/home/daifang/ncr/data/pred_yolo_100/result_predict/1000354500/RedPET
复制 1000354500_0085.png 到 RedPET 文件夹.
/export/home/daifang/ncr/data/pred_yolo_100/result_predict/1000354500/RedPET
复制 1000354500_0087.png 到 RedPET 文件夹.
/export/home/daifang/ncr/data/pred_yolo_100/result_predict/1000354500/RedPET
复制 1000354500_0091.png 到 RedPET 文件夹.
/export/home/daifang/ncr/data/pred_yolo_100/result_predict/1000354500/RedPET
复制 1000354500_0093.png 到 RedPET 文件夹.
/export/home/daifang/ncr/data/pred_yolo_100/result_predict/1000354500/RedPET
复制 10003545

KeyboardInterrupt: 

In [9]:
import torch

# 加载模型
model = torch.load('/export/home/daifang/ncr/Yolo11/ultralytics-main/runs/train/0511_PPA_CAFM2/weights/best.pt', map_location='cpu')

# 查看模型的所有层及其参数
for name, param in model.items():
    print(f"Layer: {name}, Shape: {param.shape}")


ModuleNotFoundError: No module named 'ultralytics.nn.modules.PPA'